### Code that creates X and Y coordinates and labels for point counts on an image
Updated to allow automatic infilling (October 12)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd

##### Load the image and specify starting and ending points of the grid

In [ ]:
# Read the image
img = mpimg.imread('images/O48-014.jpg') # Specify image to be loaded
um_per_pixel = 10 # Specify resolution of pixel in um per pixel
step_um = 500 # Step size of grid, in um
no_points = 600 # Specify number of desired points
start = [0.1, 0.2] # Specify x, y starting location, in relative image coordinates (0-1)
end = [0.9, 0.8] # Specify x, y starting location, in relative image coordinates (0-1)
#start = [15000,5000] # Specify x, y starting location, um from upper left
#end = [42000,27000] # Specify x, y ending location, um from upper left

y, x = img.shape[0:2]

x_um = x*um_per_pixel
y_um = y*um_per_pixel

start_um = [start[0]*img.shape[1]*um_per_pixel, start[1]*img.shape[0]*um_per_pixel]
end_um = [end[0]*img.shape[1]*um_per_pixel, end[1]*img.shape[0]*um_per_pixel]
print(start_um, end_um)


print(x_um, 'um in x-axis dimension')
print(y_um, 'um in y-axis dimension')

# Display the image with starting and ending locations
plt.imshow(img)
plt.axis('off')  # Optional: Turn off axis labels
plt.plot(start[0]*x,start[1]*y, 'o', color='white')
plt.plot(end[0]*x,end[1]*y, 's', color='white')

plt.show()

##### Create 1D arrays with X and Y coordinates and labels for each point

In [ ]:
def point_matrix(start_um, end_um, step_um, i_ini=0, reverse=False):
    # start_um, end_um are lists with starting and ending coordinates 
    # return x vals, y vales, and point count labels
    
    # Start at top-left pixel and progress to the bottom-right in snake-like pattern
    n_y_rows = int((end_um[1]-start_um[1])/step_um)+1
    n_x_cols = int((end_um[0]-start_um[0])/step_um)+1

    # Make 1D x-axis array that reflects snaking increments from top left to bottom right
    x_vals = start_um[0]+np.arange(0, n_x_cols, 1)*step_um
    X = np.tile(x_vals, n_y_rows)
    X = X.reshape(n_y_rows, n_x_cols)
    X = np.where(np.arange(len(X))[:,None]%2,X[:,::-1],X) # https://stackoverflow.com/questions/55677100/snake-traversal-of-2d-numpy-array
    X = X.flatten()

    # Make 1D y-axis values for the same array
    y_vals = start_um[1]+np.arange(0, n_y_rows, 1)*step_um
    Y = np.repeat(y_vals, n_x_cols)

    # Define 1D array with point count labels
    A = np.arange(1, n_y_rows*n_x_cols+1, 1)+i_ini
    
    if reverse:
        return np.flip(X), np.flip(Y), A
    else:
        return X, Y, A

##### Show an example of a point count matrix with a step size of 2000 um

In [ ]:
X, Y, A = point_matrix(start_um, end_um, 2000, reverse=True)

# Display the image with starting and ending locations
plt.imshow(img)
plt.axis('off')  # Optional: Turn off axis labels
plt.plot(X/um_per_pixel, Y/um_per_pixel, '-o', color='white', markersize=2)
plt.axis('off')  # Optional: Turn off axis labels
plt.plot(start[0]*x,start[1]*y, 'o', color='white')
plt.plot(end[0]*x,end[1]*y, 's', color='white')

plt.show()

##### New code below that allows one to specify the number of points desired and step size. The code will automatically infill with more points if the step size is too large to accomodate the number of desired points.

In [ ]:
num_points = 200 # Number of desired points
step_um = 8000. # Desired step size in um

c = 0 # Counter variable that records total number of points logged
d = 1. # Counter variable that reflects decreasing step count with each iteration
e = 0 # Counter variable that controls whether the snake pattern is normal or reversed

# Empty variables to store results in
Xs = []
Ys = []
As = []
legend = [] # Generation of point count

while c < num_points:
#for i in range(2):
    if e % 2 == 0: # If even
        reverse = False
    else:
        reverse = True
    # New points to add
    X, Y, A = point_matrix(start_um, end_um, step_um*d, i_ini=c, reverse=reverse)
    print(len(X))
    
    # Remove new points that overlap with existing points
    new = list(zip(X,Y))
    compare = np.isin(new, list(zip(Xs, Ys)))
    idx_to_remove = np.where(np.array([x[0] for x in compare])*np.array([x[1] for x in compare]) == 1)[0]
    new_arr = np.delete(np.asarray(new), idx_to_remove, axis=0)
    X = [x[0] for x in new_arr]
    Y = [x[1] for x in new_arr]
    A = np.arange(len(X))+c+1
    c = c + len(X)
    if c < num_points:
        Xs = Xs + list(X[:])
        Ys = Ys + list(Y[:])
        As = As + list(A[:])
        legend = legend + list(np.zeros(shape=(len(X),))+e+1)
        d = d/2.
        print('Not enough points, reducing spacing by half and continuing')
    else:
        Xs = Xs + list(X[:(num_points-c)])
        Ys = Ys + list(Y[:(num_points-c)])
        As = As + list(A[:(num_points-c)])
        legend = legend + list(np.zeros(shape=(len(X[:(num_points-c)]),))+e+1)
        print('Enough points achieved')
        break
    e = e+1

##### Plot that illustrates the point count locations

In [ ]:
color_dict = {
    1 : 'yellow',
    2 : 'green',
    3 : 'navy',
    4 : 'red'
}

symbol_dict = {
    1 : 'o',
    2 : 's',
    3 : '^',
    4 : '>'
}

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))

# Display the image with starting and ending locations
plt.imshow(img, alpha=0.5)
plt.axis('off')  # Optional: Turn off axis labels
for i in range(len(Xs)):
    #plt.plot(Xs[i]/um_per_pixel, Ys[i]/um_per_pixel, markersize=10)
    plt.plot(Xs[i]/um_per_pixel, Ys[i]/um_per_pixel, symbol_dict[legend[i]], color=color_dict[legend[i]], markersize=10)
    plt.text(x=Xs[i]/um_per_pixel, y=Ys[i]/um_per_pixel, s=As[i], color='red')
plt.axis('off')  # Optional: Turn off axis labels
#plt.plot(start[0]*x,start[1]*y, 'o', color='white')
#plt.plot(end[0]*x,end[1]*y, 's', color='white')

plt.show()

#plt.plot(X, Y, '-o')

##### Make DataFrame to export point count locations as CSV

In [ ]:
df = pd.DataFrame()
df['label'] = As
df['X (0-1)'] = np.array(Xs)/um_per_pixel/x
df['Y (0-1)'] = np.array(Ys)/um_per_pixel/y
df['X (um)'] = Xs
df['Y (um)'] = Ys

In [ ]:
df

In [ ]:
df.to_csv('point_count_locations.csv')